In [85]:
import tkinter as tk
from tkinter import filedialog
from tkinter import ttk, scrolledtext
from PIL import ImageTk, Image
import time
import datetime, os

In [86]:
Folder = ['']
file_path = ['']
file_name = ['']


### 创建主窗体

In [87]:
GUI = tk.Tk()
# GUI.withdraw()
# GUI = tk.Toplevel()       # 主窗口
GUI.title('AI图片分类')     # 窗口标题
GUI.geometry('900x700')     # 主窗口的大小
GUI.config(bg='aliceblue')

### 创建背景图片

In [88]:

# canvas = tk.Canvas(GUI, width=900, height=700, bd=0, highlightthickness=0)
# imgpath = "C:\\Users\\natlq\\Pictures\\sunflower.jpg"
# img = Image.open(imgpath)
# photo = ImageTk.PhotoImage(img)
# canvas.create_image(200, 700, image=photo)
# canvas.pack()

### Functions

In [89]:
def File_Choose():

    filename = filedialog.askopenfilename()
    filename = filename.replace('/', '\\')
    return filename

In [90]:
def Folder_Choose():
    folder_path = filedialog.askdirectory()
    folder_path = folder_path.replace('/', '\\')
    return(folder_path)

In [91]:
def start_point_get(event):
    global start_x, start_y # 声明写入全局变量
    global Canvas_Image_Source, Select_Pic_Region_Win
    Canvas_Image_Source.delete("rect1")  # 如果已经有“rect1”标记的图形，则删除
    # 在canvas 1上绘制一个矩形(rectangle是矩形的意思)
    Canvas_Image_Source.create_rectangle(event.x,
                             event.y,
                             event.x + 1,
                             event.y + 1,
                             outline="green",
                             width=5,
                             tag="rect1")
    # 在全局变量中存储坐标
    start_x, start_y = event.x, event.y

# 拖动中的事件- - - - - - - - - - - - - - - - - - - - - - - - - - -
def rect_drawing(event):
    # 拖动中的鼠标指针出现在区域外时的处理
    if event.x < 0:
        end_x = 0
    else:
        end_x = min(img_open.width, event.x)
    if event.y < 0:
        end_y = 0
    else:
        end_y = min(img_open.height, event.y)
    # 重新绘制“rect1”标签的图像
    Canvas_Image_Source.coords("rect1", start_x, start_y, end_x, end_y)

# 释放拖动时的事件- - - - - - - - - - - - - - - - - - - - - - - -
def release_action(event):
    global canvas1_size, canvas2_size, Image_Source
    canvas2_size = (400, 300)
    Img_Selected_Source =Image.open(Image_Source[0])

    # 将“rect1”标签的图像坐标恢复到原来的比例尺并获取
    start_x, start_y, end_x, end_y = [
        round(n) for n in Canvas_Image_Source.coords("rect1")
    ]
    ratio_canvas1 = img_ratio(Img_Selected_Source, canvas1_size)
    # 将所选区域恢复到实际图片尺寸
    Box = (start_x*ratio_canvas1, start_y*ratio_canvas1, end_x*ratio_canvas1, end_y*ratio_canvas1)


    Croped_Selected_Img = Img_Selected_Source.crop(Box) 
    ratio_canvas2 = img_ratio(Croped_Selected_Img, canvas2_size)
    Croped_Selected_Img_size_width, Croped_Selected_Img_size_height = Croped_Selected_Img.size
    Canvas2_Croped_Selected_Img = Croped_Selected_Img.resize((int(Croped_Selected_Img_size_width/ratio_canvas2), int(Croped_Selected_Img_size_height/ratio_canvas2)))
    print(ratio_canvas2, Box, (Croped_Selected_Img_size_width/ratio_canvas2, Croped_Selected_Img_size_height/ratio_canvas2))
    Opened_Canvas_Croped_Selected_Img = ImageTk.PhotoImage(Canvas2_Croped_Selected_Img)
    Canvas_Image_Selected_Region= tk.Canvas(Select_Pic_Region_Win, bg = 'gray')
    Canvas_Image_Selected_Region.place(x=canvas1_size[0]+2, y=0,width = canvas2_size[0], height = canvas2_size[1])
    Canvas_Image_Selected_Region.create_image(0, 0, image=Opened_Canvas_Croped_Selected_Img, anchor=tk.NW)
    Canvas_Image_Selected_Region.image = Opened_Canvas_Croped_Selected_Img

In [92]:
def Select_Pic_Region_Rectangle(canvas):
    canvas.bind("<ButtonPress-1>", start_point_get)
    canvas.bind("<Button1-Motion>", rect_drawing)
    canvas.bind("<ButtonRelease-1>", release_action)

In [93]:
def img_ratio(img, canvas_size):
    w, h = img.size
    w_ratio = w/canvas_size[0]
    h_ratio = h/canvas_size[1]
    if w_ratio > h_ratio:
        ratio = w_ratio
    else:
        ratio = h_ratio
    return ratio

In [94]:
def choosepic(image_label):
    global img_open, canvas1_size, Image_Source
    Image_Source = ['']
    path_ = filedialog.askopenfilename()
    Image_Source[0] = path_
    img_open = Image.open(path_)
    w, h = img_open.size
    ratio_canvas1 = img_ratio(img_open, canvas1_size)
    img_open = img_open.resize((int(w/ratio_canvas1), int(h/ratio_canvas1)))
    img = ImageTk.PhotoImage(img_open)
    image_label.create_image(0, 0, image=img, anchor=tk.NW)
    image_label.image = img

### 创建选择图片范围子窗体

In [95]:
def Select_Pic_Region():
      global canvas1_size , Select_Pic_Region_Win
      canvas1_size = (800, 500)
      Select_Pic_Region_Win = tk.Toplevel()
      Select_Pic_Region_Win.title('选择图片识别区域')
      Select_Pic_Region_Win.geometry('1902x1080')
      Select_Pic_Region_Win.config(bg='aliceblue')
      Select_Pic_Region_Win.focus()
      tk.Button(Select_Pic_Region_Win, text = '打开图片', font='黑体 28 bold', command=lambda: choosepic(Canvas_Image_Source)).place(x = 100, y = 830,w = 250, h = 80)
      tk.Button(Select_Pic_Region_Win, text = '选择区域', font='黑体 28 bold', command=lambda: Select_Pic_Region_Rectangle(Canvas_Image_Source)).place(x = 400, y = 830,w = 250, h = 80)
      tk.Button(Select_Pic_Region_Win, text = '确认', font='黑体 28 bold', command=lambda: Select_Pic_Region_Rectangle(Canvas_Image_Source)).place(x = 700, y = 830,w = 250, h = 80)
      global Canvas_Image_Source
      Canvas_Image_Source = tk.Canvas(Select_Pic_Region_Win, bg = 'gray')
      Canvas_Image_Source.place(x=0, y=0,width = canvas1_size[0], height = canvas1_size[1])
      # Select_Pic_Region_Win.update()


#### 创建训练子窗体

In [96]:
def Train_Win(_return1):
      Train_Win = tk.Toplevel()
      Train_Win.title('图片分类学习')
      Train_Win.geometry('900x700')
      # Train_Win.attributes("-topmost", True) # 始终在最前面显示tkinter窗口
      Train_Win.config(bg='aliceblue')
      Source_Folder = tk.LabelFrame(Train_Win, text='', padx=10, pady=10)
      Source_Folder.place(x=20, y=15)
      tk.Button(Source_Folder, text="选择源数据文件夹", font='宋体 28 bold',
            command=lambda: Folder_Choose(_return1)).pack(side="left", padx=6)
      tk.Button(Source_Folder, text="选择识别区域", font='宋体 28 bold',
            command=Select_Pic_Region).pack(side="left", padx=6)
      Start_Train = tk.LabelFrame(Train_Win, text='', padx=10, pady=10)
      Start_Train.place(x=20, y=115)
      tk.Button(Start_Train, text="开始训练", font='宋体 28 bold',
            ).pack(side="left", padx=6)
      Save_Model = tk.LabelFrame(Train_Win, text='', padx=10, pady=10)
      Save_Model.place(x=20, y=215)
      tk.Button(Save_Model, text="保存模型及识别区域", font='宋体 28 bold',
            ).pack(side="left", padx=6)

### Step1：

In [97]:
Frame_Select_Or_Train_Model = tk.LabelFrame(GUI, text='Step1:', bg='skyblue', padx=10, pady=10)
Frame_Select_Or_Train_Model.place(x=10, y=15)

In [98]:
tk.Label(Frame_Select_Or_Train_Model, text="请训练新的模型或直接进入'Step2'选择现有模型:", font='宋体 18 bold', bg='aliceblue' 
          ).pack(side="left", padx=6)
tk.Button(Frame_Select_Or_Train_Model, text='训练新的模型', font='宋体 18 bold', bg='lightyellow', 
          command=lambda: Train_Win(file_path)).pack(side='left', padx=6)

### Step2:

In [99]:
Frame_Display_Selected_Model =tk.LabelFrame(GUI, text='Step2:', bg='skyblue', padx=10, pady=10)
Frame_Display_Selected_Model.place(x=10, y=100) 

In [100]:
StringVar_Selected_Model_Name = tk.StringVar()
tk.Button(Frame_Display_Selected_Model, text="选择现有模型", font='宋体 18 bold', bg='lightyellow', 
          command=lambda: StringVar_Selected_Model_Name.set(File_Choose())).pack(side="left", padx=6)
tk.Label(Frame_Display_Selected_Model, text="选择的模型:", font='宋体 18 bold', bg='aliceblue', 
          ).pack(side="left", padx=6)
tk.Entry(Frame_Display_Selected_Model, textvariable=StringVar_Selected_Model_Name, font='宋体 10 bold', width=65,
         ).pack(side='left', padx=6)

### Step3:

In [101]:
Frame_Select_Target_Pic_Folder = tk.LabelFrame(GUI, text='Step3:', bg='skyblue', padx=10, pady=10)
Frame_Select_Target_Pic_Folder.place(x=10, y=185)

In [102]:
StringVar_Selected_Target_Folder = tk.StringVar()
tk.Button(Frame_Select_Target_Pic_Folder, text="选择待识别图片文件夹", font='宋体 18 bold', bg='lightyellow', 
          command=lambda: StringVar_Selected_Target_Folder.set(Folder_Choose())).pack(side="left", padx=6)
tk.Label(Frame_Select_Target_Pic_Folder, text="选择的文件夹:", font='宋体 18 bold', bg='aliceblue', 
          ).pack(side="left", padx=6)
tk.Entry(Frame_Select_Target_Pic_Folder, textvariable=StringVar_Selected_Target_Folder, font='宋体 10 bold', width=50
                ).pack(side='left', padx=6)

### Step4:

In [103]:
Frame_Start_Recognize = tk.LabelFrame(GUI, text='Step4:', bg='skyblue', padx=10, pady=10)
Frame_Start_Recognize.place(x=10, y=270)

In [104]:
tk.Button(Frame_Start_Recognize, text="开始识别", font='宋体 18 bold', bg='lightyellow', 
          command=Select_Pic_Region).pack(side="left", padx=6)

### 程序启动位置

In [105]:
GUI.update()
GUI.mainloop()


2.2666666666666666 (410.4, 278.4, 1080.0, 957.5999999999999) (295.5882352941177, 300.0)
2.6575 (307.2, 240.0, 1370.3999999999999, 808.8) (399.99999999999994, 214.11100658513638)
2.0633333333333335 (333.59999999999997, 408.0, 1017.5999999999999, 1027.2) (331.50242326332796, 300.0)
2.7675 (326.4, 194.4, 1432.8, 931.1999999999999) (400.0, 266.3053297199639)
3.4725 (177.6, 230.39999999999998, 1567.2, 1173.6) (400.0, 271.8502519798416)
2.0633333333333335 (355.2, 376.8, 1161.6, 996.0) (391.1147011308562, 300.0)
0.9666666666666667 (482.4, 393.59999999999997, 816.0, 684.0) (345.51724137931035, 300.0)
0.38333333333333336 (576.0, 480.0, 720.0, 595.1999999999999) (375.65217391304344, 300.0)
